In [1]:
import cv2

print("Scanning for available cameras...")

for i in range(10):
    cap = cv2.VideoCapture(i)
    if cap.isOpened():
        print(f"Camera found at index {i}")
        cap.release()

print("Camera scan complete.")

Scanning for available cameras...
Camera found at index 0
Camera found at index 1
Camera found at index 2
Camera found at index 3
Camera scan complete.


In [1]:
import cv2
import dlib
import winsound
import torch
from scipy.spatial import distance as dist
from imutils import face_utils
from deepface import DeepFace

# Function to play alarm sound
def sound_alarm():
    winsound.Beep(1000, 1000)

# Eye Aspect Ratio (EAR) function for drowsiness detection
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])  
    B = dist.euclidean(eye[2], eye[4])  
    C = dist.euclidean(eye[0], eye[3])  
    return (A + B) / (2.0 * C)

# Drowsiness detection constants
EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 48 

# Load dlib's face detector and facial landmark predictor
shape_predictor_path = r'C:\Users\Sanjay\AppData\Roaming\Microsoft\Windows\Start Menu\Programs\Python 3.12\Project\shape_predictor_68_face_landmarks.dat'
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(shape_predictor_path)

(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

# Load YOLOv5 model for pedestrian detection
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', force_reload=True, trust_repo=True)

# Initialize cameras
cap_driver = cv2.VideoCapture(1)  # Built-in camera for driver monitoring
cap_pedestrian = cv2.VideoCapture(0)  # USB camera for pedestrian detection

COUNTER_EYE = 0
ALARM_ON = False

while True:
    ret_driver, frame_driver = cap_driver.read()
    if ret_driver:
        gray_driver = cv2.cvtColor(frame_driver, cv2.COLOR_BGR2GRAY)
        rects = detector(gray_driver, 0)
        for rect in rects:
            shape = predictor(gray_driver, rect)
            shape = face_utils.shape_to_np(shape)
            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0
            (x, y, w, h) = face_utils.rect_to_bb(rect)
            cv2.rectangle(frame_driver, (x, y), (x + w, y + h), (0, 0, 255), 2)
            if ear < EYE_AR_THRESH:
                COUNTER_EYE += 1
                if COUNTER_EYE >= EYE_AR_CONSEC_FRAMES:
                    if not ALARM_ON:
                        ALARM_ON = True
                        sound_alarm()
                    cv2.putText(frame_driver, "DROWSINESS ALERT!", (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                COUNTER_EYE = 0
                ALARM_ON = False
            face_roi = frame_driver[y:y + h, x:x + w]
            try:
                result = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)
                if isinstance(result, list):
                    result = result[0]
                emotion = result['dominant_emotion']
                if emotion == "angry":
                    sound_alarm()
                    display_text = "Aggressive Driving"
                else:
                    display_text = emotion
                cv2.putText(frame_driver, display_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
            except:
                cv2.putText(frame_driver, "Emotion Detection Error", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.imshow('Driver Monitoring System', frame_driver)
    ret_pedestrian, frame_pedestrian = cap_pedestrian.read()
    if ret_pedestrian:
        results = model(frame_pedestrian)
        results.render()
        df = results.pandas().xywh[0]  
        for _, row in df.iterrows():
            if row['name'] == 'person':
                print("Pedestrian Detected!")
                sound_alarm()
        cv2.imshow('Pedestrian Detection', frame_pedestrian)
    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap_driver.release()
cap_pedestrian.release()
cv2.destroyAllWindows()

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\Sanjay/.cache\torch\hub\master.zip
YOLOv5  2025-3-10 Python-3.10.0 torch-2.6.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
